<a href="https://colab.research.google.com/github/mingyung-park/Honors-Thesis/blob/pre-experiment-checks/experiments/pre-checks/test_pairwise_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# test FOL Similarity matric (include LE)

In [1]:
!pip install datasets
!pip install huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.3 MB/s eta 0:00:00


In [2]:
from google.colab import drive
from huggingface_hub import login
from datasets import load_dataset

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
dataset = load_dataset("yuan-yang/MALLS-v0")
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.27k [00:00<?, ?B/s]

MALLS-v0.1-train.json:   0%|          | 0.00/6.36M [00:00<?, ?B/s]

MALLS-v0.1-test.json:   0%|          | 0.00/231k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/27284 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['FOL', 'NL'],
        num_rows: 27284
    })
    test: Dataset({
        features: ['FOL', 'NL'],
        num_rows: 1000
    })
})

# Consistancy calculation module

N개의 답변이 들어왔을 때, 각 답변끼리의 score를 계산하여 가장 일관성 높은 답을 선택한다.


In [4]:
%cd /content/drive/MyDrive/독립연구/codes
!pip install svgling #for visualizing parse tree
! pip install levenshtein #for levenshtein distance

/content/drive/MyDrive/독립연구/codes


In [5]:
import numpy as np
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from vec_rule_evaluator import VecRuleEvaluator
from parser import parse_fol_expression
import os

class ConsistencyCalculator:

    def __init__(self, nl_input, responses, weight=0.5, log_file="consistency_log.csv"):
        """
        Initializes the ConsistencyCalculator.

        :param nl_input: The original NL (Natural Language) input for the experiment.
        :param responses: List of FOL responses generated from the NL input.
        :param weight: Weight factor to balance LE and BLEU scores (default is 0.5).
        :param log_file: The file name where the results will be logged.
        """
        self.nl_input = nl_input
        self.responses = responses
        self.weight = weight
        self.log_file = log_file  # Set a fixed log file name

    def calculate_bleu_score(self, response1, response2):
        reference = [response1.split()]  # BLEU expects a list of references
        candidate = response2.split()
        smoothing_function = SmoothingFunction().method1
        return sentence_bleu(reference, candidate, smoothing_function=smoothing_function)

    def calculate_pairwise_scores(self):
        n = len(self.responses)
        le_scores = np.zeros((n, n))
        bleu_scores = np.zeros((n, n))
        weighted_scores = np.zeros((n, n))
        best_matches = [-1] * n

        for i in range(n):
            for j in range(i + 1, n):
                tree1 = parse_fol_expression(self.responses[i])[0]
                tree2 = parse_fol_expression(self.responses[j])[0]
                le_score, _, _ = VecRuleEvaluator.find_best_LE_score(tree1, tree2, soft_binding=False, greedy_match=True, top_n=1)
                le_scores[i, j] = le_scores[j, i] = le_score

                bleu_score = self.calculate_bleu_score(self.responses[i], self.responses[j])
                bleu_scores[i, j] = bleu_scores[j, i] = bleu_score

                weighted_score = self.weight * le_score + (1 - self.weight) * bleu_score
                weighted_scores[i, j] = weighted_scores[j, i] = weighted_score

                if weighted_scores[i, j] > weighted_scores[i, best_matches[i]]:
                    best_matches[i] = j
                if weighted_scores[j, i] > weighted_scores[j, best_matches[j]]:
                    best_matches[j] = i

        return {
            "LE": le_scores,
            "BLEU": bleu_scores,
            "Weighted": weighted_scores,
            "Best Matches": best_matches
        }

    def create_dataframe(self):
        scores = self.calculate_pairwise_scores()
        le_scores = scores["LE"]
        bleu_scores = scores["BLEU"]
        weighted_scores = scores["Weighted"]
        best_matches = scores["Best Matches"]
        n = len(self.responses)

        average_scores = weighted_scores.sum(axis=1) / (n - 1)

        # Find the most consistent response
        most_consistent_index = np.argmax(average_scores)
        most_consistent_response = self.responses[most_consistent_index]

        data = {
            "NL Input": [self.nl_input] * n,
            "FOL": self.responses,
            "Weight": [self.weight] * n,
            "Average Consistency Score": average_scores,
            "Best Match (FOL Index)": best_matches,
            "Best Match (FOL Expression)": [self.responses[idx] if idx != -1 else None for idx in best_matches],
            "Best Match LE Score": [le_scores[i, best_matches[i]] if best_matches[i] != -1 else None for i in range(n)],
            "Best Match BLEU Score": [bleu_scores[i, best_matches[i]] if best_matches[i] != -1 else None for i in range(n)],
            "Best Match Weighted Score": [weighted_scores[i, best_matches[i]] if best_matches[i] != -1 else None for i in range(n)],
            "Most Consistent Response": [most_consistent_response] * n,
            "Most Consistent Index": [most_consistent_index] * n
        }

        df = pd.DataFrame(data)
        return df

    def run_consistency_evaluation(self):
        df = self.create_dataframe()

        # Check if the log file already exists
        file_exists = os.path.isfile(self.log_file)

        # Append the DataFrame to the CSV file if it exists, otherwise create a new file
        df.to_csv(self.log_file, mode='a', index=False, header=not file_exists)

        print(df)
        return df


In [6]:
# First experiment
nl_input1 = "If a person is happy, then they smile."
responses1 = [
    "∀x (P(x) → Q(x))",
    "∀y (P(y) → Q(y))",
    "∃z (P(z) ∧ Q(z))",
    "∀x (¬P(x) ∨ Q(x))"
]

calculator1 = ConsistencyCalculator(nl_input1, responses1, weight=0.7)
df1 = calculator1.run_consistency_evaluation()

# Second experiment with a new NL input
nl_input2 = "If it rains, then the ground gets wet."
responses2 = [
    "∀a (R(a) → W(a))",
    "∀b (R(b) → W(b))",
    "∃c (R(c) ∧ W(c))",
    "∀d (¬R(d) ∨ W(d))"
]

calculator2 = ConsistencyCalculator(nl_input2, responses2, weight=0.7)
df2 = calculator2.run_consistency_evaluation()


                                 NL Input                FOL  Weight  \
0  If a person is happy, then they smile.   ∀x (P(x) → Q(x))     0.7   
1  If a person is happy, then they smile.   ∀y (P(y) → Q(y))     0.7   
2  If a person is happy, then they smile.   ∃z (P(z) ∧ Q(z))     0.7   
3  If a person is happy, then they smile.  ∀x (¬P(x) ∨ Q(x))     0.7   

   Average Consistency Score  Best Match (FOL Index)  \
0                   0.600922                       3   
1                   0.591368                       0   
2                   0.350000                       0   
3                   0.592888                       0   

  Best Match (FOL Expression)  Best Match LE Score  Best Match BLEU Score  \
0           ∀x (¬P(x) ∨ Q(x))                  1.0               0.095544   
1            ∀x (P(x) → Q(x))                  1.0               0.080343   
2            ∀x (P(x) → Q(x))                  0.5               0.000000   
3            ∀x (P(x) → Q(x))                  1.0

In [11]:
import pandas as pd

log_file = "consistency_log.csv"
try:
    df = pd.read_csv(log_file)
    print("Log file loaded successfully.")

    df['Sub Index'] = df.groupby('NL Input').cumcount()
    df.set_index(['NL Input', 'Sub Index'], inplace=True)

    def view_logs_for_nl(nl_input):
        if nl_input not in df.index.get_level_values('NL Input'):
            print(f"No logs found for NL input: '{nl_input}'")
        else:
            filtered_df = df.loc[nl_input]
            print(f"Logs for NL input: '{nl_input}'")
            print(filtered_df)

    # Example usage
    # nl_input_to_view = "If a person is happy, then they smile."
    # view_logs_for_nl(nl_input_to_view)

except FileNotFoundError:
    print(f"The log file '{log_file}' does not exist.")


Log file loaded successfully.


In [10]:
df

FOL  Weight  \
NL Input                               Sub Index                              
If a person is happy, then they smile. 0           ∀x (P(x) → Q(x))     0.7   
                                       1           ∀y (P(y) → Q(y))     0.7   
                                       2           ∃z (P(z) ∧ Q(z))     0.7   
                                       3          ∀x (¬P(x) ∨ Q(x))     0.7   
If it rains, then the ground gets wet. 0           ∀a (R(a) → W(a))     0.7   
                                       1           ∀b (R(b) → W(b))     0.7   
                                       2           ∃c (R(c) ∧ W(c))     0.7   
                                       3          ∀d (¬R(d) ∨ W(d))     0.7   

                                                  Average Consistency Score  \
NL Input                               Sub Index                              
If a person is happy, then they smile. 0                           0.600922   
                                       1                           0.591368   
                                       2                           0.350000   
                                       3                           0.592888   
If it rains, then the ground gets wet. 0                           0.591368   
                                       1                           0.591368   
                                       2                           0.350000   
                                       3                           0.583333   

                                                  Best Match (FOL Index)  \
NL Input                               Sub Index                           
If a person is happy, then they smile. 0                               3   
                                       1                               0   
                                       2                               0   
                                       3                               0   
If it rains, then the ground gets wet. 0                               1   
                                       1                               0   
                                       2                               0   
                                       3                               0   

                                                 Best Match (FOL Expression)  \
NL Input                               Sub Index                               
If a person is happy, then they smile. 0                   ∀x (¬P(x) ∨ Q(x))   
                                       1                    ∀x (P(x) → Q(x))   
                                       2                    ∀x (P(x) → Q(x))   
                                       3                    ∀x (P(x) → Q(x))   
If it rains, then the ground gets wet. 0                    ∀b (R(b) → W(b))   
                                       1                    ∀a (R(a) → W(a))   
                                       2                    ∀a (R(a) → W(a))   
                                       3                    ∀a (R(a) → W(a))   

                                                  Best Match LE Score  \
NL Input                               Sub Index                        
If a person is happy, then they smile. 0                          1.0   
                                       1                          1.0   
                                       2                          0.5   
                                       3                          1.0   
If it rains, then the ground gets wet. 0                          1.0   
                                       1                          1.0   
                                       2                          0.5   
                                       3                          1.0   

                                                  Best Match BLEU Score  \
NL Input                               Sub Index                          
If a person is happy, then they smile. 